## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec
import os

In [3]:
lvl1 = 'Corpus of Presential Speeches/'

speech_dict = {'president': [], 'text': []}

for folder in os.listdir(lvl1):
    if folder != '.DS_Store':
        for file in os.listdir(lvl1 + folder):
            with open(f'{lvl1}{folder}/{file}', 'r') as f:
                data = f.read().replace('\n\n', '\n')
                paragraphs = data.split('\n')
    #             if 'date=' in data:
                for paragraph in paragraphs[2:]:
                    speech_dict['president'].append(folder)
                    speech_dict['text'].append(paragraph)
                
df = pd.DataFrame.from_dict(speech_dict)
print(df.shape)
df.head()

(36501, 2)


,president,text
0,coolidge,We have come here to dedicate a cornerstone th...
1,coolidge,It is but natural that such a design should be...
2,coolidge,"Next to him will come Thomas Jefferson, whose ..."
3,coolidge,"After our country had been established, enlarg..."
4,coolidge,That the principles for which these three men ...


In [5]:
df['length'] = df['text'].apply(lambda x: len(x.split(' ')))
df = df[df.length > 3]
df = df[~df.text.str.contains("<")]
df.head()

,president,text,length
0,coolidge,We have come here to dedicate a cornerstone th...,85
1,coolidge,It is but natural that such a design should be...,73
2,coolidge,"Next to him will come Thomas Jefferson, whose ...",96
3,coolidge,"After our country had been established, enlarg...",77
4,coolidge,That the principles for which these three men ...,61


In [6]:
target = df.president
data = df['text'].map(word_tokenize).values

In [7]:
total_vocabulary = set(word.lower() for text in data for word in text)
print(f'Total Words in Vocabulary: {len(total_vocabulary)}')

Total Words in Vocabulary: 38788


In [9]:
glove = {}
with open('glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [10]:
n = np.random.randint(0, len(total_vocabulary)-10)

for k in list(glove.keys())[n:n+10]:
    print(f'{k}: {glove[k][:5]}')
    print(f'Vector Length: {len(glove[k])}\n')

compete: [-0.42036   0.040774  0.3123    0.76219  -0.36937 ]
Vector Length: 50

membership: [-0.2466    0.89564  -0.048209 -0.62159   0.50642 ]
Vector Length: 50

electoral: [-0.36981  -0.7491    0.096122  0.52829  -0.29578 ]
Vector Length: 50

purpose: [ 0.29487  0.44688 -0.34654 -0.15737  1.0373 ]
Vector Length: 50

multiple: [ 0.83276  0.59926  0.68273  0.74114 -0.5631 ]
Vector Length: 50

computers: [ 0.56105  -1.1966    2.4125    0.35548  -0.046729]
Vector Length: 50

changing: [-0.27312  -0.07375  -0.1799   -0.30478   0.083617]
Vector Length: 50

agenda: [ 0.1657   0.3202  -0.19835 -0.32254  0.37216]
Vector Length: 50

satellite: [ 1.1241   0.61814  2.0402   1.2708  -0.41115]
Vector Length: 50

resort: [ 1.7924  1.2216 -1.207   1.3854 -1.0688]
Vector Length: 50



In [11]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note from Mike: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # It can't be used in a sklearn Pipeline. 
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf  = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr  = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Logistic Regression', LogisticRegression())])

In [13]:
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]

In [68]:
import warnings
warnings.filterwarnings('ignore')

scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model in models]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished


In [69]:
scores

[('Random Forest', 0.2600893874176554),
 ('Support Vector Machine', 0.19063518718284356),
 ('Logistic Regression', 0.22400822299117315)]

In [70]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

In [71]:
y = pd.get_dummies(target).values

In [72]:
tokenizer = text.Tokenizer(num_words=30000)
tokenizer.fit_on_texts(list(df.text))
list_tokenized_headlines = tokenizer.texts_to_sequences(df.text)
X_t = sequence.pad_sequences(list_tokenized_headlines, maxlen=10000)

In [73]:
len(np.unique(target))

44

In [79]:
np.unique(target)

array(['Trump', 'adams', 'arthur', 'bharrison', 'buchanan', 'bush',
       'carter', 'cleveland', 'clinton', 'coolidge', 'eisenhower',
       'fdroosevelt', 'fillmore', 'ford', 'garfield', 'grant', 'gwbush',
       'harding', 'harrison', 'hayes', 'hoover', 'jackson', 'jefferson',
       'johnson', 'jqadams', 'kennedy', 'lbjohnson', 'lincoln', 'madison',
       'mckinley', 'monroe', 'nixon', 'obama', 'pierce', 'polk', 'reagan',
       'roosevelt', 'taft', 'taylor', 'truman', 'tyler', 'vanburen',
       'washington', 'wilson'], dtype=object)

In [80]:
embedding_size = 128
input_ = Input(shape=(10000,))
x = Embedding(30000, embedding_size)(input_)
x = Bidirectional(LSTM(50, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(.25)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(.25)(x)
x = Dense(44, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

In [81]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 10000, 128)        3840000   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 10000, 100)        71600     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 100)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_8 (Dropout)          (None, 50)                0         
__________

In [83]:
model.fit(X_t, y, epochs=5, batch_size=8, validation_split=0.1)

Train on 30530 samples, validate on 3393 samples
Epoch 1/5
   80/30530 [..............................] - ETA: 7:54:17 - loss: 3.7806 - acc: 0.0125

KeyboardInterrupt: 